<a href="https://colab.research.google.com/github/Saannrr/ML-AI-GDSC-UG-2021/blob/main/ML_AI_GDSC_UG_Week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [5]:
ratings.head()

,movie_id,user_id,user_rating
0,357,138,4.0
1,709,92,2.0
2,412,301,4.0
3,56,60,4.0
4,895,197,3.0


In [9]:
n_movie = ratings.movie_id.nunique()
n_user = ratings.user_id.nunique()

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, test_size=0.2)

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten

EMBEDDING_DIM = 35

# input layer
movie_input = Input(shape=1)
user_input = Input(shape=1)

#embedding layer
movie_embedding = Embedding(n_movie+1, EMBEDDING_DIM)(movie_input)
user_embedding = Embedding(n_user+1, EMBEDDING_DIM)(user_input)

movie_flat = Flatten()(movie_embedding)
user_flat = Flatten()(user_embedding)

# output layer
output = Dot(1)([movie_flat, user_flat])

model = Model([movie_input, user_input], [output])

In [26]:
model.compile(optimizer='adam', loss='mse')

In [27]:
model.fit(x=[train.movie_id, train.user_id], y=train.user_rating,
          epochs=30, 
          batch_size=128)

Epoch 1/30
625/625 [==============================] - 2s 2ms/step - loss: 12.1663
Epoch 2/30
625/625 [==============================] - 1s 2ms/step - loss: 2.6596
Epoch 3/30
625/625 [==============================] - 1s 2ms/step - loss: 1.1465
Epoch 4/30
625/625 [==============================] - 1s 2ms/step - loss: 0.9473
Epoch 5/30
625/625 [==============================] - 1s 2ms/step - loss: 0.8778
Epoch 6/30
625/625 [==============================] - 1s 2ms/step - loss: 0.8400
Epoch 7/30
625/625 [==============================] - 1s 2ms/step - loss: 0.8145
Epoch 8/30
625/625 [==============================] - 1s 2ms/step - loss: 0.7938
Epoch 9/30
625/625 [==============================] - 1s 2ms/step - loss: 0.7757
Epoch 10/30
625/625 [==============================] - 1s 2ms/step - loss: 0.7591
Epoch 11/30
625/625 [==============================] - 1s 2ms/step - loss: 0.7415
Epoch 12/30
625/625 [==============================] - 1s 2ms/step - loss: 0.7229
Epoch 13/30
625/625 [===

In [28]:
model.evaluate(x=[test.movie_id, test.user_id], y=test.user_rating)

625/625 [==============================] - 1s 1ms/step - loss: 0.9500


0.9499675631523132

In [29]:
model.save('model.h5')

In [31]:
import numpy as np
import matplotlib.pyplot as plt

def get_recommendations(user_id, movies, model):
  movies = movies.copy()
  user_ids = np.array([user_id] * len(movies))
  results = model([movies.movie_id.values, user_ids]).numpy().reshape(-1)

  movies['predicted_rating'] = pd.Series(results)
  movies = movies.sort_values('predicted_rating', ascending=False)

  print(f'Recommendations for user {user_id}')
  return movies

get_recommendations(140, movies, model)

Recommendations for user 140


,movie_id,movie_title,movie_genres_str,predicted_rating
1404,1405,Boy's Life 2 (1997),['Documentary'],5.154822
268,269,"Full Monty, The (1997)","[""Children's""]",5.062214
115,116,Cold Comfort Farm (1995),"[""Children's""]",4.992833
1357,1358,The Deadly Cure (1996),['unknown'],4.903272
112,113,"Horseman on the Roof, The (Hussard sur le toit...",['Documentary'],4.899982
...,...,...,...,...
1320,1321,Open Season (1996),"[""Children's""]",-0.090982
1595,1596,Nemesis 2: Nebula (1995),"['unknown', 'Sci-Fi', 'Thriller']",-0.092259
1594,1595,Shopping (1994),"['unknown', 'Thriller']",-0.099592
1200,1201,Marlene Dietrich: Shadow and Light (1996),['Crime'],-0.128418


In [32]:
def get_embedding(movies, model):
  vectors = model.layers[2](movies.movie_id.values).numpy()
  vectors = pd.DataFrame(vectors)
  vectors.to_csv('vectors.tsv', sep='\t', header=False)

  movies.to_csv('meta.tsv', sep='\t')

get_embedding(movies, model)